# Day 3

In [ ]:
import pykx as kx

🩺 After days of exposure to this unknown environment, you begin to notice changes in your physical state. Luckily, a wearable vitals monitor survived the crash. It has recorded sporadic biometric readings like heart rate and hydration levels.

To understand the impact of your environment on your health, you decide to align your vitals data with the weather conditions recorded in Day 1 — and visualize how your body reacted.

📂 Data
You now have two datasets:

weather (from day 1)

| time                    | sensorId | temperatureF | humidity |
|-------------------------|----------|--------------|----------|
| 2026-01-25 00:00:00.000 | 67       | 74.49463     | 62.2822  |
| 2026-01-25 00:00:00.500 | 91       | 73.88359     | 62.53392 |
| 2026-01-25 00:00:01.000 | 42       | 72.1795      | 62.53017 |
| 2026-01-25 00:00:01.500 | 84       | 72.90408     | 64.90296 |
| ...                     |          |              |          |

vitals.csv with heart rate and hydration readings every few seconds to few minutes

| time                   | heartRate | hydration |
|------------------------|-----------|-----------|
| 2026.01.25D00:01:47.00 | 50        | 59.68     |
| 2026.01.25D00:06:22.00 | 55        | 66.99     |
| 2026.01.25D00:08:13.00 | 58        | 67.44     |
| 2026.01.25D00:09:29.00 | 55        | 63.28     |
| ...                    |           |           |

🧩 Challenge

Into a q instance load:
- weather data from day 1
- vitals.csv into a table called vitals with data types:
    - time as timestamp
    - heartRate as integer
    - hydration as real

- For each vitals reading, find the closest previous weather observation.
- Connecting to your q instance using Python, or using another KDB-X integration, plot a scatter plot of heart rate vs temperature.
- Using your visualization or otherwise, identify periods where heart rate exceeds 100 bpm and temperature is above 77°F.

In [ ]:
%%py

## Extra exercise

Dehydration is making you lose your minds, and you're arguing over design concepts. Will you be able to figure out which type of join is appropriate in each hypothetical situation?

1. Create a table that shows all nick names from the sensor table and their cost from the cost table. (all sensors must appear)
2. Create a table that shows just the sensors with a known location.

In [ ]:
/1. Basic sensor Table
show sensor:([] id:1 2 3 4; nickName:`Alice`Bob`Charlie`Dave)

/2. Keyed Cost Table
show cost:([id:1 2 3] cost:50000 60000 70000)

/3. Location Table
show loc:([] id:1 2; city:`London`NY)

3. Now someone is saying that it is not posible to select the sensors that have a cost smaller than the average without doing a join but, is it not? ([hint](https://code.kx.com/q/ref/fby/))


In [ ]:
sensors2:([] 
    id:1 2 3 4 5 6; 
    city:`London`NY`London`NY`London`NY; 
    cost:50000 60000 75000 45000 55000 40000
 )

## Reference

[Q Datatypes](https://code.kx.com/q/basics/datatypes/)

[qsql](https://code.kx.com/q/basics/qsql/)

[joins](https://code.kx.com/q/basics/joins/)

[parquet](https://code.kx.com/kdb-x/modules/parquet/introduction.html) API

[PyKX](https://code.kx.com/pykx/3.1)

Operators:
- [0:](https://code.kx.com/q/ref/file-text/#load-csv) - File reading
- [#](https://code.kx.com/q/ref/take/) - Take
- [xasc](https://code.kx.com/q/ref/asc/#xasc)
- [aj](https://code.kx.com/q/ref/aj/) - As of join